In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            for item, label in zip(data, labels):
                if label in [0, 1]:
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

(10000, 3072) (10000,)
(2000, 3072) (2000,)
[[ 0.63921569  0.6627451   0.60784314 ...  0.6         0.5372549
   0.6       ]
 [ 0.36470588  0.16078431  0.22352941 ... -0.74901961 -0.25490196
  -0.17647059]
 [-0.1372549  -0.14509804 -0.14509804 ... -0.70980392 -0.74901961
  -0.70980392]
 ...
 [-1.         -1.         -1.         ... -0.3254902  -0.41176471
  -0.11372549]
 [-0.76470588 -0.42745098 -0.2        ... -0.83529412 -0.60784314
  -0.12941176]
 [-0.7254902  -0.73333333 -0.73333333 ...  0.70196078  0.71764706
   0.74117647]] [0 1 0 0 1 0 0 1 1 0]


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
# [None]
y = tf.placeholder(tf.float64, [None])

# (3072, 1)
w = tf.get_variable('w', [x.get_shape()[-1], 1],
                   initializer=tf.random_normal_initializer(0, 1))

# (1, )
b = tf.get_variable('b', [1],
                   initializer=tf.constant_initializer(0.0))

# [None, 3072] * [3072, 1] = [None, 1]
y_ = tf.matmul(x, w) + b

# [None, 1]
p_y_1 = tf.nn.sigmoid(y_)
# [None, 1]
y_reshaped = tf.reshape(y, (-1, 1))
y_reshaped_float = tf.cast(y_reshaped, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

# bool
predict = p_y_1 > 0.5
# [1,0,1,0]
correct_prediction = tf.equal(tf.cast(predict, tf.float64), y_reshaped)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [5]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps =100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if i % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if i % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels
                                       })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Train] step: 0, loss: 0.49594, acc:0.50000
(2000, 3072) (2000,)
[Test] Step: 1, acc: 0.38400
[Train] step: 500, loss: 0.19872, acc:0.80000
[Train] step: 1000, loss: 0.08715, acc:0.90000
[Train] step: 1500, loss: 0.20201, acc:0.80000
[Train] step: 2000, loss: 0.15320, acc:0.85000
[Train] step: 2500, loss: 0.19967, acc:0.80000
[Train] step: 3000, loss: 0.34265, acc:0.65000
[Train] step: 3500, loss: 0.20000, acc:0.80000
[Train] step: 4000, loss: 0.15003, acc:0.85000
[Train] step: 4500, loss: 0.20019, acc:0.80000
[Train] step: 5000, loss: 0.20001, acc:0.80000
(2000, 3072) (2000,)
[Test] Step: 5001, acc: 0.80950
[Train] step: 5500, loss: 0.21316, acc:0.75000
[Train] step: 6000, loss: 0.33133, acc:0.65000
[Train] step: 6500, loss: 0.16976, acc:0.80000
[Train] step: 7000, loss: 0.20000, acc:0.80000
[Train] step: 7500, loss: 0.18899, acc:0.80000
[Train] step: 8000, loss: 0.14741, acc:0.85000
[Train] step: 8500, loss: 0.29969, acc:0.70000
[Train] step: 9000, loss: 0.13672, acc:0.80000
[Train] 

[Train] step: 77000, loss: 0.10010, acc:0.90000
[Train] step: 77500, loss: 0.10001, acc:0.90000
[Train] step: 78000, loss: 0.20212, acc:0.80000
[Train] step: 78500, loss: 0.24868, acc:0.75000
[Train] step: 79000, loss: 0.05041, acc:0.95000
[Train] step: 79500, loss: 0.10125, acc:0.90000
[Train] step: 80000, loss: 0.00001, acc:1.00000
(2000, 3072) (2000,)
[Test] Step: 80001, acc: 0.82350
[Train] step: 80500, loss: 0.20030, acc:0.80000
[Train] step: 81000, loss: 0.10175, acc:0.90000
[Train] step: 81500, loss: 0.05066, acc:0.95000
[Train] step: 82000, loss: 0.15048, acc:0.85000
[Train] step: 82500, loss: 0.14875, acc:0.85000
[Train] step: 83000, loss: 0.05000, acc:0.95000
[Train] step: 83500, loss: 0.10409, acc:0.90000
[Train] step: 84000, loss: 0.25022, acc:0.75000
[Train] step: 84500, loss: 0.17627, acc:0.80000
[Train] step: 85000, loss: 0.05008, acc:0.95000
(2000, 3072) (2000,)
[Test] Step: 85001, acc: 0.82250
[Train] step: 85500, loss: 0.19996, acc:0.80000
[Train] step: 86000, loss: 0